In [1]:
import pandas as pd
pd.options.display.max_columns = 100

import sys, os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from util.params import ENRICH_ALPHA, MULTI_HYP_CORR_METHOD

In [2]:
assoc_muts_df = pd.read_pickle("./data/11_df.pkl")
display(assoc_muts_df.shape, assoc_muts_df.head())

(114, 45)

,index,Details,mutation target annotation,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,isolate,presence,tech_rep,coding,range,gene RegulonDB ID,genetic features,oriC,pseudogene,TFBS,promoter,RBS,attenuator terminator,terminator,genetic,genomic features,genetic feature links,operons,operon links,regulators,regulator links,pathways,pathway links,COGs,COG links,temperature,carbon-source,supplement,strain-description,taxonomy-id,base-media,nitrogen-source,phosphorous-source,sulfur-source,calcium-source
2527,28,None,[crl],DEL,257908,NC_000913,Δ776 bp,1,TOL_putrescine,50,1,1.0,1,True,"(257908, 258683)","{ECK120001080, ECK125229291, ECK125229290}","[{'name': 'insA9', 'RegulonDB ID': 'ECK1252292...",False,False,{},{},{},{},{},True,"[{'name': 'insB9', 'RegulonDB ID': 'ECK1252292...","{'ECK125229291': ['ECK125229291'], 'ECK1252292...",[],{},[],{},[],{},[],{},37 celsius,glucose(2),putrescine,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
2528,3,coding (163/210 nt),cspC,INS,1907273,NC_000913,(CGTCCTG)1→2,1,TOL_putrescine,50,1,1.0,1,True,"(1907273, 1907273)",{ECK120002092},"[{'name': 'cspC', 'RegulonDB ID': 'ECK12000209...",False,False,{},{},{},{},{},True,"[{'name': 'cspC', 'RegulonDB ID': 'ECK12000209...",{'ECK120002092': ['ECK120002092']},"[{'name': 'yobF-cspC', 'RegulonDB ID': 'ECK120...",{'ECK120028784': ['ECK120002092']},[],{},[],{},"[{'name': 'Transcription', 'mutation set count...",{'Transcription': ['ECK120028784']},37 celsius,glucose(2),putrescine,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
2529,29,None,"insB1, insA",DEL,1978503,NC_000913,Δ776 bp,1,TOL_putrescine,50,1,1.0,1,True,"(1978503, 1979278)","{ECK120023915, ECK120003617, ECK120003616}","[{'name': 'insB-5', 'RegulonDB ID': 'ECK120003...",False,False,"{ECK120017150, ECK120013472}","{ECK125095446, ECK120034252}",{},{},{},True,"[{'name': 'flhDC TFBS', 'RegulonDB ID': 'ECK12...","{'ECK120003616': ['ECK120013472', 'ECK12000361...","[{'name': 'flhDC', 'RegulonDB ID': 'ECK1200149...","{'ECK120014934': ['ECK120003616', 'ECK12000361...","[{'name': 'YjjQ', 'significantly associated co...","{'YjjQ': ['flhDC', 'flhDC'], 'Fur': ['flhDC', ...",[],{},"[{'name': 'Mobilome: prophages, transposons', ...","{'Mobilome: prophages, transposons': ['ECK1250...",37 celsius,glucose(2),putrescine,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
2530,32,intergenic (‑1/+1),gatC/gatC,DEL,2173363,NC_000913,Δ2 bp,1,TOL_putrescine,50,1,1.0,1,False,"(2173363, 2173364)",{ECK120002276},"[{'name': 'gatC', 'RegulonDB ID': 'ECK12000227...",False,False,{},{},{},{},{},False,"[{'name': 'gatC', 'RegulonDB ID': 'ECK12000227...",{'ECK120002276': ['ECK120002276']},"[{'name': 'gatYZABCD', 'RegulonDB ID': 'ECK120...",{'ECK120015849': ['ECK120002276']},"[{'name': 'GatR', 'significantly associated co...","{'GatR': ['gatYZABCD'], 'ArcA': ['gatYZABCD'],...",[],{},[],{},37 celsius,glucose(2),putrescine,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
2531,4,N34K (AAT→AAG),mreB,SNP,3400986,NC_000913,A→C,1,TOL_putrescine,50,1,1.0,1,True,"(3400986, 3400986)",{ECK120000601},"[{'name': 'mreB', 'RegulonDB ID': 'ECK12000060...",False,False,{},{},{},{},{},True,"[{'name': 'mreB', 'RegulonDB ID': 'ECK12000060...",{'ECK120000601': ['ECK120000601']},"[{'name': 'mreBCD', 'RegulonDB ID': 'ECK120030...",{'ECK120030764': ['ECK120000601']},[],{},[],{},"[{'name': 'Cell cycle control, cell division, ...","{'Cell cycle control, cell division, chromosom...",37 celsius,glucose(2),putrescine,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)


# Significance

In [3]:
comp_mut_df = pd.DataFrame(columns=["observed mutation count"])
for i, r in assoc_muts_df.iterrows():
    for f, links in r["pathway links"].items():
        if f not in comp_mut_df.index:
            df = pd.DataFrame({"observed mutation count": len(links)}, index=[f])
            comp_mut_df = comp_mut_df.append(df)
        else:
            comp_mut_df.loc[f, "observed mutation count"] += len(links)

comp_mut_df

,observed mutation count
Purine metabolism,12
Pyrimidine metabolism,10
Pentose phosphate pathway,4
Pentose and glucuronate interconversions,2
Arginine and proline metabolism,3
Glyoxylate and dicarboxylate metabolism,2
Drug metabolism - other enzymes,3
Cysteine and methionine metabolism,1
Lysine biosynthesis,1
Tyrosine metabolism,1


In [4]:
for _, r in assoc_muts_df.iterrows():
    for feat_d in r["pathways"]:
        feat_d["mutation set count"] = comp_mut_df.loc[feat_d["name"]]["observed mutation count"]

In [5]:
pathway_len_df = pd.read_pickle("./data/pathway_len_df.pkl")
pathway_len_df.head()

,pathway length
pathway_name,
"Glycine, serine and threonine metabolism",54804
Cysteine and methionine metabolism,36129
Lysine biosynthesis,18189
Vitamin B6 metabolism,10138
Pentose phosphate pathway,51305


In [6]:
import numpy as np

if len(comp_mut_df):
    comp_mut_df["length"] = comp_mut_df.apply(lambda r: pathway_len_df.loc[r.index, "pathway length"])
    total_seq_len = np.sum(comp_mut_df["length"])
    comp_mut_df["proportion"] = comp_mut_df["length"].apply(lambda comp_len: comp_len/total_seq_len)
    comp_mut_df

In [7]:
from collections import Counter

if len(comp_mut_df):
    comp_mut_df['equal or larger count'] = 0
    component_l = comp_mut_df.index.tolist()
    proportion_l = comp_mut_df["proportion"].tolist()
    num_muts = np.sum(comp_mut_df["observed mutation count"])

    # slides from UW speak about the relationship of permutations and precision.
    # should read through them to get a sense of what is the least iterations that should execute.
    num_itr = 10000
    for i in range(0, num_itr):
        random_mut_comp_a = np.random.choice(
            a=component_l,
            size=int(num_muts),
            p=proportion_l,
            replace=True,
        )
        random_mut_comp_count_d = Counter(random_mut_comp_a)
        for component in random_mut_comp_count_d.keys():
            if random_mut_comp_count_d[component] >= comp_mut_df.loc[component, 'observed mutation count']:
                comp_mut_df.loc[component, 'equal or larger count'] += 1

    comp_mut_df["p value"] = comp_mut_df["equal or larger count"].apply(
        lambda count: count/num_itr)

    comp_mut_df["significant"] = comp_mut_df["p value"].apply(lambda p_val: True if p_val < ENRICH_ALPHA else False)

    display(comp_mut_df.head())

,observed mutation count,length,proportion,equal or larger count,p value,significant
Purine metabolism,12,154034,0.146006,1835,0.1835,False
Pyrimidine metabolism,10,89371,0.084713,374,0.0374,True
Pentose phosphate pathway,4,51305,0.048631,3497,0.3497,False
Pentose and glucuronate interconversions,2,32217,0.030538,5671,0.5671,False
Arginine and proline metabolism,3,71864,0.068119,8013,0.8013,False


In [8]:
from statsmodels.stats import multitest


if len(comp_mut_df):
    pvals = comp_mut_df["p value"]
    rejects, pvals_corrected, alphacSidak, alphacBonf = multitest.multipletests(
        pvals=pvals,
        alpha=ENRICH_ALPHA,
        method=MULTI_HYP_CORR_METHOD)

    pd.options.display.max_rows = 500
    comp_mut_df["corrected p value"] = pvals_corrected
    comp_mut_df["corrected significance"] = rejects
    comp_mut_df.head()

In [9]:
# # not wanting to consider mutation events with only 1 features as being significant
if len(comp_mut_df):
    comp_mut_df["final significance"] = comp_mut_df.apply(lambda r: r["corrected significance"] if r["observed mutation count"] > 1 else False, axis=1)
# comp_mut_df.head()

In [10]:
if len(comp_mut_df):
    for i, r in assoc_muts_df.iterrows():
        for d in r["pathways"]:
            if d["name"] in comp_mut_df.index:
                d["significantly enriched"] = comp_mut_df.loc[d["name"]]["final significance"]

In [11]:
assert(len(assoc_muts_df.exp.unique())==1)
exp = assoc_muts_df.exp.unique()[0]
comp_mut_df.to_csv("./data/supp/permutation_test_results_per_experiment/"+exp+"_mut_pwy_signif.csv")

In [12]:
assoc_muts_df.to_pickle("./data/12_df.pkl")

In [13]:
# import seaborn as sns
# import matplotlib
# import matplotlib.pyplot as plt
# %matplotlib inline
# plt.rcParams["figure.dpi"] = 100
# sns.set_context("talk")
# sns.set_style("ticks")


# def _get_name_signif_from_name(name, json):
#     is_signif = False
#     for d in json:
#         if d["name"] == name:
#             is_signif = d["significant"]
#             break
#     return is_signif


# # The reason we're using the links dictionary is because we want to count how genomic features were mutated for the genetic target.
# targ_cnt_d = dict()
# for _, r in assoc_muts_df.iterrows():
#     link_d = r["pathway links"]
#     for targ_name, hits in link_d.items():
#         is_signif = _get_name_signif_from_name(targ_name, r["pathways"])
#         t = str(targ_name)
#         if is_signif:
#             t = '*' + t
#         if t not in targ_cnt_d.keys():
#             targ_cnt_d[t] = 0
#         targ_cnt_d[t] += len(hits)
        
# df = pd.DataFrame.from_dict(targ_cnt_d, orient='index', columns=["mutated feature count"])
# df = df.sort_values("mutated feature count")
# df = df[-20:]
# ax = df.plot.barh(y='mutated feature count',
# #                   figsize=(10,25),
#                   figsize=(5,10),
#                   color="#4C72B0",
#                   width=1
#                  )
# for i, v in enumerate(df["mutated feature count"]):
#     ax.text(v, i - 0.2, str(v))
# sns.despine(ax=ax, top=True, right=True)
# ax.set_title("Top 20 mutated pathways")
# ax.get_legend().remove()

In [14]:
# import seaborn as sns
# import matplotlib
# import matplotlib.pyplot as plt
# %matplotlib inline
# plt.rcParams["figure.dpi"] = 100
# sns.set_context("talk")
# sns.set_style("ticks")


# def _get_name_signif_from_name(name, json):
#     is_signif = False
#     for d in json:
#         if d["name"] == name:
#             is_signif = d["significant"]
#             break
#     return is_signif


# # The reason we're using the links dictionary is because we want to count how genomic features were mutated for the genetic target.
# targ_cnt_d = dict()
# for _, r in assoc_muts_df.iterrows():
#     link_d = r["pathway links"]
#     for targ_name, hits in link_d.items():
#         is_signif = _get_name_signif_from_name(targ_name, r["pathways"])
#         t = str(targ_name)
#         if is_signif:
#             t = '*' + t
#             if t not in targ_cnt_d.keys():
#                 targ_cnt_d[t] = 0
#             targ_cnt_d[t] += len(hits)
            
# if len(targ_cnt_d) > 0:
#     df = pd.DataFrame.from_dict(targ_cnt_d, orient='index', columns=["mutated feature count"])
#     df = df.sort_values("mutated feature count")
#     df = df[-20:]
#     display(df)
#     ax = df.plot.barh(y='mutated feature count',
#     #                   figsize=(10,25),
#                       figsize=(5,10),
#                       color="#4C72B0",
#                       width=1
#                      )
#     for i, v in enumerate(df["mutated feature count"]):
#         ax.text(v, i - 0.2, str(v))
#     sns.despine(ax=ax, top=True, right=True)
#     ax.set_title("Significant mutated pathways")
#     ax.get_legend().remove()